# Safe exec

Run a script in exec function in a safe manner.

> In the example below, it allows the user to import `os` (which is not safe)
> just to show how to implement.

In [16]:
import typing as _
import importlib
import builtins
import types

class Dummy:
    """Just to not break the script"""
    def __init__(self, name: str) -> None:
        self.name = name

    def __repr__(self) -> str:
        return f'<Dummy {self.name!r} object>'

    def __call__(self, *args, **kwargs) -> _.Self:
        print(f'[ERROR] Call {self.name!r} not allowed')
        return self

    def __getattr__(self, item) -> _.Self:
        print(f'[ERROR] GetAttr {self.name!r} not allowed')
        return self

    def __getitem__(self, item) -> _.Self:
        print(f'[ERROR] Getitem {self.name!r} not allowed')
        return self


def my_import(name, globals=None, locals=None, fromlist=(), level=0):
    if name in ('os',):
        return importlib.__import__(name, globals, locals, fromlist, level)

    return Dummy(name)


def my_builtins():
    module_name = "my_builtins"
    dynamic_module = types.ModuleType(module_name)

    def custom_str(value):
        return f'The value {value!r}'

    dynamic_module.str = custom_str
    dynamic_module.print = builtins.print
    dynamic_module.__import__ = my_import

    return dynamic_module


def exec_script(script: str) -> dict:
    _globals = {'__builtins__': my_builtins()}
    _locals = {}

    exec(script, _globals, _locals)

    return _locals


In [17]:
# Everything in the script is fine
l = exec_script('def hello(name):\n'
               '    print(f"Hello {name}!")')
l['hello']('Arnaldo')

Hello Arnaldo!


In [18]:
# sys is not allowed, but it will not rise any error
l = exec_script('import sys\n'
                'sys.argsv[0]\n'
                'print(str(10_000))')

[ERROR] GetAttr 'sys' not allowed
[ERROR] Getitem 'sys' not allowed
The value 10000


In [19]:
# repr is not allowed and it rises an error
try:
    l = exec_script('d = {1: 11, 2:22}\n'
                    'd = repr(d)')
except NameError as ex:
    print(f'Failed: {ex}')

Failed: name 'repr' is not defined
